# Benchmarks

## Initialize

In [1]:
%load_ext autoreload
%autoreload 2

import os
import math
import pathlib
import numpy as np
import pandas as pd
import pyarrow as pa
from tqdm.auto import tqdm
from IPython.display import clear_output

import warnings
from lifelines.utils import CensoringType
from lifelines.utils import concordance_index

In [2]:
node = !hostname
if "sc" in node[0]:
    base_path = "/sc-projects/sc-proj-ukb-cvd"
else: 
    base_path = "/data/analysis/ag-reils/ag-reils-shared/cardioRS"
print(base_path)

project_label = "22_medical_records"
project_path = f"{base_path}/results/projects/{project_label}"
figure_path = f"{project_path}/figures"
output_path = f"{project_path}/data"

pathlib.Path(figure_path).mkdir(parents=True, exist_ok=True)
pathlib.Path(output_path).mkdir(parents=True, exist_ok=True)

tag = 220306

/sc-projects/sc-proj-ukb-cvd


In [3]:
import ray
ray.init(num_cpus=24)#, dashboard_port=24762, dashboard_host="0.0.0.0", include_dashboard=True)#, webui_url="0.0.0.0"))

2022-03-07 15:48:14,972	INFO services.py:1374 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '10.32.105.6',
 'raylet_ip_address': '10.32.105.6',
 'redis_address': '10.32.105.6:6379',
 'object_store_address': '/tmp/ray/session_2022-03-07_15-48-08_961689_1049722/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-03-07_15-48-08_961689_1049722/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2022-03-07_15-48-08_961689_1049722',
 'metrics_export_port': 57179,
 'gcs_address': '10.32.105.6:44129',
 'node_id': 'f7df8c83d7788fc92b9c3e21921d6efdb71856a90390f861748da4aa'}

In [4]:
endpoint_dict = {'OMOP_4306655': 'OMOP_4306655', 'phecode_401': 'phecode_401 - Hypertension', 'phecode_401-1': 'phecode_401-1 - Essential hypertension', 'phecode_130': 'phecode_130 - Cancer (solid tumor, excluding BCC)', 'phecode_089': 'phecode_089 - Infections', 'phecode_460': 'phecode_460 - Acute respiratory infection', 'phecode_202': 'phecode_202 - Diabetes mellitus', 'phecode_202-2': 'phecode_202-2 - Type 2 diabetes', 'phecode_404': 'phecode_404 - Ischemic heart disease', 'phecode_583': 'phecode_583 - Chronic kidney disease', 'phecode_089-2': 'phecode_089-2 - Viral infections', 'phecode_718': 'phecode_718 - Back pain', 'phecode_708': 'phecode_708 - Osteoarthritis', 'phecode_475': 'phecode_475 - Asthma', 'phecode_089-1': 'phecode_089-1 - Bacterial infections', 'phecode_809': 'phecode_809 - Pain', 'phecode_713': 'phecode_713 - Symptoms related to joints', 'phecode_416': 'phecode_416 - Cardiac arrhythmia and conduction disorders', 'phecode_239': 'phecode_239 - Hyperlipidemia', 'phecode_713-3': 'phecode_713-3 - Pain in joint', 'phecode_460-1': 'phecode_460-1 - Acute upper respiratory infection', 'phecode_522': 'phecode_522 - Gastrointestinal inflammation', 'phecode_474': 'phecode_474 - Chronic obstructive pulmonary disease [COPD]', 'phecode_460-2': 'phecode_460-2 - Acute lower respiratory infection', 'phecode_660': 'phecode_660 - Infection of the skin', 'phecode_239-1': 'phecode_239-1 - Hypercholesterolemia', 'phecode_105': 'phecode_105 - Malignant neoplasm of the breast', 'phecode_120': 'phecode_120 - Hemo onc - by cell of origin', 'phecode_239-11': 'phecode_239-11 - Pure hypercholesterolemia', 'phecode_116': 'phecode_116 - Secondary malignant neoplasm', 'phecode_416-2': 'phecode_416-2 - Atrial fibrillation and flutter', 'phecode_520': 'phecode_520 - Hernia', 'phecode_200': 'phecode_200 - Disorders of thyroid gland', 'phecode_286': 'phecode_286 - Mood [affective] disorders', 'phecode_841': 'phecode_841 - Drug and medical agent allergy', 'phecode_101': 'phecode_101 - Malignant neoplasm of the digestive organs', 'phecode_527': 'phecode_527 - Abdominal pain', 'phecode_908': 'phecode_908 - Edema, proteinuria and hypertensive disorders in pregnancy, childbirth and the puerperium', 'phecode_403': 'phecode_403 - Angina pectoris', 'phecode_200-1': 'phecode_200-1 - Hypothyroidism', 'phecode_404-2': 'phecode_404-2 - Coronary atherosclerosis [Atherosclerotic heart disease]', 'phecode_988': 'phecode_988 - Family history of cardiovascular disease', 'phecode_529': 'phecode_529 - Symptoms involving digestive system', 'phecode_281': 'phecode_281 - Substance abuse, dependence, and withdrawl', 'phecode_716': 'phecode_716 - Intervertebral disc disorder', 'phecode_200-14': 'phecode_200-14 - Hypothyroidism, not specified as secondary', 'phecode_181': 'phecode_181 - Autoimmune disease', 'phecode_282': 'phecode_282 - Nicotine dependence (current and history of)', 'phecode_282-1': 'phecode_282-1 - Current tobacco use and nicotine dependence', 'phecode_292': 'phecode_292 - Somatoform disorders', 'phecode_286-2': 'phecode_286-2 - Major depressive disorder', 'phecode_164': 'phecode_164 - Anemia', 'phecode_401-3': 'phecode_401-3 - Hypertensive chronic kidney disease', 'phecode_523': 'phecode_523 - Diverticular disease [Bowel diverticulosis]', 'phecode_281-2': 'phecode_281-2 - Substance dependence', 'phecode_089-3': 'phecode_089-3 - Fungal infections', 'phecode_511': 'phecode_511 - Gastro-esophageal reflux disease', 'phecode_809-3': 'phecode_809-3 - Pain in limb', 'phecode_401-2': 'phecode_401-2 - Hypertensive heart disease', 'phecode_390': 'phecode_390 - Disorders of external ear', 'phecode_908-1': 'phecode_908-1 - Preeclampsia and eclampsia', 'phecode_841-1': 'phecode_841-1 - Allergy to other anti-infective agents', 'phecode_908-2': 'phecode_908-2 - Pre-existing hypertension complicating pregnancy, childbirth and the puerperium', 'phecode_801': 'phecode_801 - Cough', 'phecode_005': 'phecode_005 - Mycobacteria', 'phecode_005-1': 'phecode_005-1 - Mycobacterium tuberculosis', 'phecode_627': 'phecode_627 - Menopausal and postmenopausal disorders', 'phecode_520-2': 'phecode_520-2 - Diaphragmatic hernia [Hiatal hernia]', 'phecode_404-1': 'phecode_404-1 - Myocardial infarction [Heart attack]', 'phecode_371': 'phecode_371 - Cataract', 'phecode_510': 'phecode_510 - Diseases of esophagus', 'phecode_052': 'phecode_052 - Herpesvirus', 'phecode_061': 'phecode_061 - Influenza virus', 'phecode_292-3': 'phecode_292-3 - Pain disorders related to psychological factors', 'phecode_112': 'phecode_112 - Malignant neoplasm of other and ill-defined sites', 'phecode_668': 'phecode_668 - Dermatitis [eczema]', 'phecode_718-4': 'phecode_718-4 - Low back pain', 'phecode_591': 'phecode_591 - Urinary tract infection', 'phecode_841-11': 'phecode_841-11 - Penicillin allergy', 'phecode_331': 'phecode_331 - Headache', 'phecode_120-1': 'phecode_120-1 - Myeloid', 'phecode_236': 'phecode_236 - Overweight and obesity', 'phecode_236-1': 'phecode_236-1 - Obesity', 'phecode_136': 'phecode_136 - Benign neoplasm of the digestive organs', 'phecode_723': 'phecode_723 - Enthesopathy-Enthesitis-Tendinopathy', 'phecode_247': 'phecode_247 - Disorders of mineral metabolism and mineral deficiencies', 'phecode_469': 'phecode_469 - Bronchitis', 'phecode_424': 'phecode_424 - Heart failure', 'phecode_594': 'phecode_594 - Abnormality of urination', 'phecode_436': 'phecode_436 - Atherosclerosis [ASCVD]', 'phecode_840': 'phecode_840 - Allergy', 'phecode_660-6': 'phecode_660-6 - Cellulitis and abscess', 'phecode_523-2': 'phecode_523-2 - Diverticula of colon', 'phecode_679': 'phecode_679 - Skin symptoms', 'phecode_247-7': 'phecode_247-7 - Disorders of iron metabolism', 'phecode_136-4': 'phecode_136-4 - Benign neoplasm of colon, rectum, anus and anal canal', 'phecode_715': 'phecode_715 - Non-inflammatory spondylopathy', 'phecode_462': 'phecode_462 - Sinusitis', 'phecode_477': 'phecode_477 - Inhalation lung injury', 'phecode_909': 'phecode_909 - Diabetes mellitus or abnormal glucose in pregnancy, childbirth, and the puerperium', 'phecode_468': 'phecode_468 - Pneumonia', 'phecode_116-1': 'phecode_116-1 - Secondary malignancy of lymph nodes', 'phecode_101-4': 'phecode_101-4 - Malignant neoplasm of the colon and rectum', 'phecode_660-1': 'phecode_660-1 - Fungal infection of the skin', 'phecode_335': 'phecode_335 - Nerve root and plexus disorders', 'phecode_510-2': 'phecode_510-2 - Esophagitis', 'phecode_596': 'phecode_596 - Other disorders of bladder', 'phecode_424-1': 'phecode_424-1 - Left heart failure', 'phecode_721': 'phecode_721 - Synoviopathy and bursopathy', 'phecode_413': 'phecode_413 - Heart valve disorders', 'phecode_678': 'phecode_678 - Other skin and connective tissue disorders', 'phecode_522-9': 'phecode_522-9 - Gastritis', 'phecode_095': 'phecode_095 - Sequela of infection', 'phecode_557': 'phecode_557 - Gastrointestinal hemorrhage', 'phecode_052-1': 'phecode_052-1 - Herpes simplex', 'phecode_520-1': 'phecode_520-1 - Hernia of the abdominal wall', 'phecode_336': 'phecode_336 - Mononeuropathies', 'phecode_160': 'phecode_160 - Nutritional anemias', 'phecode_465': 'phecode_465 - Pharyngitis', 'phecode_108': 'phecode_108 - Malignant neoplasm of the urinary tract', 'phecode_705': 'phecode_705 - Rheumatoid arthritis and other inflammatory polyarthropathies', 'phecode_487': 'phecode_487 - Hemorrhage from respiratory passages', 'phecode_367': 'phecode_367 - Inflammation of the eye', 'phecode_488': 'phecode_488 - Abnormalities of breathing', 'phecode_583-1': 'phecode_583-1 - End stage renal disease [CDK, stage 5]', 'phecode_465-1': 'phecode_465-1 - Acute Pharyngitis', 'phecode_481': 'phecode_481 - Interstitial pulmonary diseases', 'phecode_529-5': 'phecode_529-5 - Constipation', 'phecode_390-1': 'phecode_390-1 - Otitis externa', 'phecode_462-1': 'phecode_462-1 - Acute sinusitis', 'phecode_103': 'phecode_103 - Malignant neoplasm of the skin', 'phecode_088': 'phecode_088 - Sexually transmitted disease', 'phecode_469-2': 'phecode_469-2 - Chronic bronchitis', 'phecode_107': 'phecode_107 - Malignant neoplasm of male genitalia', 'phecode_070': 'phecode_070 - Candidiasis', 'phecode_136-41': 'phecode_136-41 - Benign neoplasm of the colon', 'phecode_600': 'phecode_600 - Benign prostatic hyperplasia', 'phecode_120-2': 'phecode_120-2 - Lymphoid', 'phecode_107-2': 'phecode_107-2 - Malignant neoplasm of the prostate', 'phecode_331-6': 'phecode_331-6 - Migraine', 'phecode_440': 'phecode_440 - Embolism and thrombosis', 'phecode_477-2': 'phecode_477-2 - Hypersensitivity pneumonitis', 'phecode_256': 'phecode_256 - Disorders of fluid, electrolyte and acid-base balance', 'phecode_439': 'phecode_439 - Hemorrhoids', 'phecode_550': 'phecode_550 - Disorders of the gallbladder', 'phecode_374': 'phecode_374 - Disorders of the retina', 'phecode_123': 'phecode_123 - Multiple myeloma and malignant plasma cell neoplasms', 'phecode_472': 'phecode_472 - Diseases of vocal cords and larynx, not elsewhere classified', 'phecode_417': 'phecode_417 - Abnormalities of heart beat', 'phecode_981': 'phecode_981 - Family history of malignant neoplasm', 'phecode_526': 'phecode_526 - Intestinal infection', 'phecode_288': 'phecode_288 - Anxiety disorders', 'phecode_466': 'phecode_466 - Tonsillitis and adenoiditis', 'phecode_608': 'phecode_608 - Other disorders of male genital organs', 'phecode_160-1': 'phecode_160-1 - Iron deficiency anemia', 'phecode_247-72': 'phecode_247-72 - Iron deficiency', 'phecode_819': 'phecode_819 - General symptoms and other findings', 'phecode_724': 'phecode_724 - Other symptoms and disorders of the soft tissue', 'phecode_202-1': 'phecode_202-1 - Type 1 diabetes', 'phecode_840-8': 'phecode_840-8 - Allergies related to other diseases-symptoms', 'phecode_122': 'phecode_122 - Lymphoma', 'phecode_708-7': 'phecode_708-7 - Generalized osteoarthritis', 'phecode_333': 'phecode_333 - Sleep disorders', 'phecode_715-1': 'phecode_715-1 - Spondylosis', 'phecode_164-1': 'phecode_164-1 - Microcytic anemia', 'phecode_120-21': 'phecode_120-21 - Mature B-cell', 'phecode_052-3': 'phecode_052-3 - Varicella zoster virus', 'phecode_084': 'phecode_084 - Parasites', 'phecode_487-3': 'phecode_487-3 - Hemoptysis', 'phecode_391': 'phecode_391 - Disorders of the middle ear', 'phecode_909-2': 'phecode_909-2 - Pre-existing diabetes mellitus in pregnancy*', 'phecode_116-4': 'phecode_116-4 - Secondary malignant neoplasm of liver', 'phecode_396': 'phecode_396 - Hearing impairment', 'phecode_522-1': 'phecode_522-1 - Inflammatory bowel disease', 'phecode_682': 'phecode_682 - Other follicular disorders', 'phecode_466-1': 'phecode_466-1 - Acute tonsillitis and adenoiditis', 'phecode_477-1': 'phecode_477-1 - Pneumoconiosis', 'phecode_660-12': 'phecode_660-12 - Dermatophytosis', 'phecode_708-1': 'phecode_708-1 - Primary osteoarthritis', 'phecode_331-8': 'phecode_331-8 - Headache NOS', 'phecode_444': 'phecode_444 - Venous insufficiency', 'phecode_144': 'phecode_144 - Gynecological benign neoplasms', 'phecode_354': 'phecode_354 - Dizziness and giddiness', 'phecode_367-1': 'phecode_367-1 - Conjunctivitis', 'phecode_524': 'phecode_524 - Functional intestinal disorder', 'phecode_481-1': 'phecode_481-1 - Pulmonary eosinophilia', 'phecode_550-1': 'phecode_550-1 - Gallstones [Cholelithiasis]', 'phecode_052-32': 'phecode_052-32 - Herpes zoster', 'phecode_472-4': 'phecode_472-4 - Laryngeal spasm', 'phecode_444-1': 'phecode_444-1 - Varicose veins', 'phecode_390-4': 'phecode_390-4 - Impacted cerumen', 'phecode_586': 'phecode_586 - Other disorders of the kidney and ureters', 'phecode_807': 'phecode_807 - Malaise and fatigue', 'phecode_324': 'phecode_324 - Extrapyramidal and movement disorders', 'phecode_463': 'phecode_463 - Rhinitis and nasal congestion', 'phecode_524-1': 'phecode_524-1 - Irritable bowel syndrome', 'phecode_101-41': 'phecode_101-41 - Malignant neoplasm of the colon', 'phecode_627-4': 'phecode_627-4 - Menorrhagia-Excessive and frequent menstruation', 'phecode_138': 'phecode_138 - Benign neoplasm of the skin', 'phecode_123-1': 'phecode_123-1 - Multiple myeloma', 'phecode_375': 'phecode_375 - Abnormal intraocular pressure', 'phecode_116-6': 'phecode_116-6 - Secondary malignancy of bone', 'phecode_444-11': 'phecode_444-11 - Varicose veins of lower extremities', 'phecode_122-2': 'phecode_122-2 - Non-Hodgkin lymphoma', 'phecode_718-2': 'phecode_718-2 - Cervicalgia', 'phecode_102': 'phecode_102 - Malignant neoplasm of the thoracic and respiratory organs', 'phecode_582': 'phecode_582 - Acute kidney failure', 'phecode_703': 'phecode_703 - Chrystal arthropathies', 'phecode_705-1': 'phecode_705-1 - Rheumatoid arthritis', 'phecode_448': 'phecode_448 - Peripheral vascular disease', 'phecode_291': 'phecode_291 - Dissociative, conversion and factitious disorders', 'phecode_593': 'phecode_593 - Hematuria', 'phecode_330': 'phecode_330 - Epilepsy, recurrent seizures, convulsions', 'phecode_619': 'phecode_619 - Noninflammatory female genital disorders', 'phecode_679-1': 'phecode_679-1 - Rash and other nonspecific skin eruption', 'phecode_703-1': 'phecode_703-1 - Hyperuricemia', 'phecode_718-5': 'phecode_718-5 - Sciatica', 'phecode_800': 'phecode_800 - Chest pain', 'phecode_626': 'phecode_626 - Disorders of menstruation and other abnormal bleeding from female genital tract', 'phecode_280': 'phecode_280 - Substance related disorders', 'phecode_703-11': 'phecode_703-11 - Gout', 'phecode_330-1': 'phecode_330-1 - Epilepsy', 'phecode_700': 'phecode_700 - Diffuse diseases of connective tissue', 'phecode_520-11': 'phecode_520-11 - Inguinal hernia', 'phecode_010': 'phecode_010 - Corynebacterium', 'phecode_402': 'phecode_402 - Elevated blood pressure reading without diagnosis of hypertension', 'phecode_528-2': 'phecode_528-2 - Vomiting', 'phecode_528-1': 'phecode_528-1 - Nausea', 'phecode_528': 'phecode_528 - Nausea and vomiting', 'phecode_121': 'phecode_121 - Leukemia', 'phecode_670': 'phecode_670 - Seborrheic keratosis', 'phecode_116-2': 'phecode_116-2 - Secondary malignancy of respiratory organs', 'phecode_280-1': 'phecode_280-1 - Alcohol use disorders', 'phecode_665': 'phecode_665 - Psoriasis', 'phecode_627-1': 'phecode_627-1 - Postmenopausal bleeding', 'phecode_557-8': 'phecode_557-8 - Hemorrhage of rectum and anus', 'phecode_513': 'phecode_513 - Peptic ulcer', 'phecode_431': 'phecode_431 - Stroke and transient cerebral ischemic attacks', 'phecode_722': 'phecode_722 - Fasciopathy', 'phecode_350': 'phecode_350 - Other symptoms involving nervous system', 'phecode_144-2': 'phecode_144-2 - Benign neoplasms of the uterus', 'phecode_529-1': 'phecode_529-1 - Diarrhea', 'phecode_488-1': 'phecode_488-1 - Dyspnea [Shortness of breath]', 'phecode_004': 'phecode_004 - Streptococcus', 'phecode_308': 'phecode_308 - Signs and symptoms involving emotional state', 'phecode_594-3': 'phecode_594-3 - Urinary incontinence', 'phecode_232': 'phecode_232 - Vitamin deficiencies', 'phecode_800-2': 'phecode_800-2 - Precordial pain', 'phecode_102-1': 'phecode_102-1 - Malignant neoplasm of the of bronchus and lung', 'phecode_983': 'phecode_983 - Family history of other endocrine and metabolic diseases', 'phecode_106': 'phecode_106 - Gynecological malignant neoplasms', 'phecode_585': 'phecode_585 - Kidney stone disease', 'phecode_375-1': 'phecode_375-1 - Glaucoma', 'phecode_360': 'phecode_360 - Inflammation of eyelids', 'phecode_404-11': 'phecode_404-11 - Acute myocardial infarction', 'phecode_709': 'phecode_709 - Acquired deformities of fingers and toes', 'phecode_336-1': 'phecode_336-1 - Carpal tunnel syndrome', 'phecode_440-1': 'phecode_440-1 - Venous thromboembolism', 'phecode_286-21': 'phecode_286-21 - Major depressive disorder, recurrent', 'phecode_808': 'phecode_808 - Syncope and collapse', 'phecode_618': 'phecode_618 - Genital prolapse', 'phecode_592': 'phecode_592 - Cystitis and urethritis', 'phecode_416-4': 'phecode_416-4 - Heart block', 'phecode_333-1': 'phecode_333-1 - Sleep apnea', 'phecode_542': 'phecode_542 - Chronic liver disease and sequelae', 'phecode_950': 'phecode_950 - Puerperal infection or fever', 'phecode_413-2': 'phecode_413-2 - Aortic valve disorders', 'phecode_916': 'phecode_916 - Infections and carrier state in pregnancy', 'phecode_486': 'phecode_486 - Other respiratory disorders', 'phecode_916-1': 'phecode_916-1 - Infections of genitourinary tract in pregnancy', 'phecode_462-2': 'phecode_462-2 - Chronic sinusitis', 'phecode_702': 'phecode_702 - Infective and reactive arthropathies', 'phecode_592-1': 'phecode_592-1 - Cystitis', 'phecode_483': 'phecode_483 - Pleural effusion', 'phecode_530': 'phecode_530 - Disease of anus and rectum', 'phecode_419': 'phecode_419 - Presence of cardiac device', 'phecode_413-1': 'phecode_413-1 - Mitral valve disorders', 'phecode_465-2': 'phecode_465-2 - Chronic Pharyngitis', 'phecode_709-2': 'phecode_709-2 - Acquired deformities of toe', 'phecode_471': 'phecode_471 - Other disorders of nose and nasal sinuses', 'phecode_050': 'phecode_050 - Enterovirus', 'phecode_626-1': 'phecode_626-1 - Irregular menstrual cycle-bleeding', 'phecode_682-1': 'phecode_682-1 - Cutaneous cyst', 'phecode_673': 'phecode_673 - Skin changes due to chronic exposure to nonionizing radiation', 'phecode_448-9': 'phecode_448-9 - Peripheral vascular disease NOS [includes PAD]', 'phecode_468-9': 'phecode_468-9 - Lobar pneumonia*', 'phecode_416-21': 'phecode_416-21 - Atrial fibrillation', 'phecode_092': 'phecode_092 - Bacteremia, Sepsis, and SIRS', 'phecode_202-4': 'phecode_202-4 - Other specified diabetes*', 'phecode_290': 'phecode_290 - Reaction to severe stress, and adjustment disorders', 'phecode_476': 'phecode_476 - Bronchiectasis', 'phecode_050-5': 'phecode_050-5 - Enteroviral vesicular pharyngitis', 'phecode_092-2': 'phecode_092-2 - Sepsis', 'phecode_673-1': 'phecode_673-1 - Actinic keratosis', 'phecode_711': 'phecode_711 - Disorder of patella', 'phecode_660-2': 'phecode_660-2 - Bacterial infection of the skin', 'phecode_006': 'phecode_006 - Neisseria', 'phecode_627-2': 'phecode_627-2 - Symptomatic menopause', 'phecode_662': 'phecode_662 - Rosacea', 'phecode_139': 'phecode_139 - Benign sarcoma-related cancers', 'phecode_446': 'phecode_446 - Hypotension', 'phecode_006-2': 'phecode_006-2 - Neisseria gonorrhea', 'phecode_391-1': 'phecode_391-1 - Otitis media', 'phecode_610': 'phecode_610 - Benign mammary dysplasias', 'phecode_328': 'phecode_328 - Dementias and cerebral degeneration', 'phecode_522-12': 'phecode_522-12 - Ulcerative colitis', 'phecode_294': 'phecode_294 - Sexual dysfunction and disorders', 'phecode_350-5': 'phecode_350-5 - Repeated falls*', 'phecode_469-1': 'phecode_469-1 - Acute bronchitis', 'phecode_525': 'phecode_525 - Intestinal malabsorption', 'phecode_479': 'phecode_479 - Pulmonary insufficiency and acute respiratory distress syndrome', 'phecode_416-211': 'phecode_416-211 - Paroxysmal atrial fibrillation*', 'phecode_727': 'phecode_727 - Other disorders of bone', 'phecode_351': 'phecode_351 - Disturbances of skin sensation', 'phecode_625': 'phecode_625 - Pain and other symptoms associated with female genital organs', 'phecode_716-3': 'phecode_716-3 - Spinal disc displacement (herniation)', 'phecode_723-4': 'phecode_723-4 - Lateral epicondylitis (Tennis elbow)', 'phecode_116-3': 'phecode_116-3 - Secondary malignant neoplasm of digestive systems', 'phecode_431-1': 'phecode_431-1 - Stroke', 'phecode_594-1': 'phecode_594-1 - Retention of urine', 'phecode_336-5': 'phecode_336-5 - Mononeuritis of lower limb', 'phecode_056': 'phecode_056 - Human papillomavirus', 'phecode_108-5': 'phecode_108-5 - Malignant neoplasm of the bladder', 'phecode_132': 'phecode_132 - Sequelae of cancer', 'phecode_200-13': 'phecode_200-13 - Postprocedural hypothyroidism', 'phecode_829': 'phecode_829 - Nonspecific findings on examination of blood', 'phecode_812': 'phecode_812 - Edema', 'phecode_463-2': 'phecode_463-2 - Allergic rhinitis', 'phecode_463-21': 'phecode_463-21 - Seasonal allergic rhinitis', 'phecode_395': 'phecode_395 - Other diseases of inner ear', 'phecode_800-1': 'phecode_800-1 - Chest pain on breathing', 'phecode_417-1': 'phecode_417-1 - Palpitations', 'phecode_979': 'phecode_979 - Transplated organ', 'phecode_721-1': 'phecode_721-1 - Synovitis and tenosynovitis', 'phecode_679-4': 'phecode_679-4 - Pruritus', 'phecode_136-42': 'phecode_136-42 - Benign neoplasm of rectum and anus', 'phecode_614': 'phecode_614 - Inflammatory diseases of female pelvic organs', 'phecode_682-12': 'phecode_682-12 - Pilar and trichodermal cyst', 'phecode_230': 'phecode_230 - Malnutrition and underweight', 'phecode_926': 'phecode_926 - Mental disorders complicating pregnancy, childbirth and the puerperium', 'phecode_124': 'phecode_124 - Myeloproliferative disorder', 'phecode_613': 'phecode_613 - Other nonmalignant breast conditions', 'phecode_286-3': 'phecode_286-3 - Premenstrual dysphoric disorder', 'phecode_144-21': 'phecode_144-21 - Leiomyoma of uterus', 'phecode_507': 'phecode_507 - Lesions of mouth', 'phecode_581': 'phecode_581 - Renal tubulo-interstitial diseases', 'phecode_926-1': 'phecode_926-1 - Postpartum mood disturbance*', 'phecode_926-11': 'phecode_926-11 - Postpartum depression*', 'phecode_286-4': 'phecode_286-4 - Dysthymic disorder', 'phecode_711-1': 'phecode_711-1 - Derangement of meniscus', 'phecode_362': 'phecode_362 - Other disorders of the eyelids', 'phecode_324-1': 'phecode_324-1 - Parkinsonism', 'phecode_726': 'phecode_726 - Osteoporosis and low bone density', 'phecode_400': 'phecode_400 - Rheumatic fever and chronic rheumatic heart diseases', 'phecode_721-5': 'phecode_721-5 - Bursitis', 'phecode_686': 'phecode_686 - Chronic ulcer of skin', 'phecode_440-13': 'phecode_440-13 - Phlebitis and thrombophlebitis', 'phecode_433': 'phecode_433 - Other cerebrovascular disease', 'phecode_363': 'phecode_363 - Disorders of lacrimal system', 'phecode_707': 'phecode_707 - Other arthropathies', 'phecode_724-1': 'phecode_724-1 - Myalgia', 'phecode_324-11': "phecode_324-11 - Parkinson's disease (Primary)", 'phecode_597': 'phecode_597 - Other disorders of urethra and urinary tract', 'phecode_717': 'phecode_717 - Other and unspecified dorsopathies', 'phecode_337': 'phecode_337 - Polyneuropathies', 'phecode_371-31': 'phecode_371-31 - Age-related nuclear cataract', 'phecode_371-3': 'phecode_371-3 - Nuclear cataract', 'phecode_676': 'phecode_676 - Hypertrophic conditions of skin', 'phecode_512': 'phecode_512 - Dysphagia', 'phecode_516': 'phecode_516 - Other diseases of stomach and duodenum', 'phecode_400-2': 'phecode_400-2 - Chronic rheumatic heart diseases', 'phecode_170-1': 'phecode_170-1 - Neutropenia', 'phecode_170': 'phecode_170 - Decreased white blood cell count', 'phecode_522-11': "phecode_522-11 - Crohn's disease", 'phecode_709-21': 'phecode_709-21 - Hallux valgus (Bunion)', 'phecode_841-4': 'phecode_841-4 - Allergy to analgesic agent', 'phecode_177': 'phecode_177 - Abnormality of the lymph nodes', 'phecode_700-9': 'phecode_700-9 - Other diffuse diseases of connective tissue', 'phecode_584': 'phecode_584 - Renal failure', 'phecode_326': 'phecode_326 - Demyelinating diseases of the central nervous system', 'phecode_599': 'phecode_599 - Other symptoms-disorders or the urinary system', 'phecode_329': 'phecode_329 - Symptoms and signs involving cognitive functions and awareness', 'phecode_426': 'phecode_426 - Other heart disorders in diseases NEC', 'phecode_142-1': 'phecode_142-1 - Lump or mass in breast', 'phecode_142': 'phecode_142 - Lump or mass in breast or nonspecific abnormal breast exam', 'phecode_200-7': 'phecode_200-7 - Iodine-deficiency related thyroid disorders*', 'phecode_618-1': 'phecode_618-1 - Prolapse of vaginal walls', 'phecode_499': 'phecode_499 - Other symptoms and disorders of the respiratory system', 'phecode_510-8': "phecode_510-8 - Barrett's esophagus", 'phecode_504': 'phecode_504 - Periodontal diseases', 'phecode_138-2': 'phecode_138-2 - Melanocytic nevi*', 'phecode_841-12': 'phecode_841-12 - Allergy to antibiotic agent (excluding penicillin)', 'phecode_016': 'phecode_016 - Chlamydia', 'phecode_414': 'phecode_414 - Cardiomyopathy', 'phecode_325': 'phecode_325 - Symptoms and signs related to movement disorders', 'phecode_706': 'phecode_706 - Other inflammatory spondylopathies', 'phecode_230-2': 'phecode_230-2 - Abnormal loss of weight and underweight', 'phecode_232-2': 'phecode_232-2 - Vitamin B group deficiency', 'phecode_326-1': 'phecode_326-1 - Multiple sclerosis', 'phecode_230-21': 'phecode_230-21 - Abnormal weight loss', 'phecode_239-2': 'phecode_239-2 - Hyperglyceridemia', 'phecode_622': 'phecode_622 - Polyp of female genital organs', 'phecode_594-32': 'phecode_594-32 - Stress incontinence', 'phecode_239-3': 'phecode_239-3 - Mixed hyperlipidemia', 'phecode_139-5': 'phecode_139-5 - Lipoma', 'phecode_473': 'phecode_473 - Other diseases of upper respiratory tract', 'phecode_106-3': 'phecode_106-3 - Malignant neoplasm of the ovary', 'phecode_467': 'phecode_467 - Laryngitis and tracheitis', 'phecode_545': 'phecode_545 - Nonspecific abnormal results of function study of liver', 'phecode_431-11': 'phecode_431-11 - Cerebral infarction [Ischemic stroke]', 'phecode_668-1': 'phecode_668-1 - Atopic dermatitis', 'phecode_526-2': 'phecode_526-2 - Viral enteritis', 'phecode_715-4': 'phecode_715-4 - Spinal stenosis', 'phecode_336-51': 'phecode_336-51 - Lesion of sciatic nerve', 'phecode_374-5': 'phecode_374-5 - Macular degeneration', 'phecode_394': 'phecode_394 - Disorders of vestibular function', 'phecode_121-1': 'phecode_121-1 - Acute leukemia', 'phecode_101-42': 'phecode_101-42 - Malignant neoplasm of the rectum', 'phecode_722-1': 'phecode_722-1 - Plantar fascial fibromatosis [Plantar fasciitis]', 'phecode_240': 'phecode_240 - Disorders of amino-acid transport and metabolism', 'phecode_719': 'phecode_719 - Disorders of muscle', 'phecode_121-2': 'phecode_121-2 - Chronic leukemia', 'phecode_240-1': 'phecode_240-1 - Disorders of aromatic amino-acid metabolism', 'phecode_514': 'phecode_514 - Gastrointestinal obstruction', 'phecode_604': 'phecode_604 - Disorders of penis', 'phecode_386': 'phecode_386 - Visual disturbances', 'phecode_440-3': 'phecode_440-3 - Pulmonary embolism', 'phecode_325-2': 'phecode_325-2 - Abnormality of gait and mobility', 'phecode_467-1': 'phecode_467-1 - Acute laryngitis and tracheitis', 'phecode_392': 'phecode_392 - Otalgia and effusion of ear', 'phecode_614-5': 'phecode_614-5 - Inflammatory disease of cervix, vagina, and vulva', 'phecode_976': 'phecode_976 - Adverse effect of anesthesia', 'phecode_683': 'phecode_683 - Nail disorders', 'phecode_705-3': 'phecode_705-3 - Polymyalgia rheumatica', 'phecode_526-1': 'phecode_526-1 - Bacterial enteritis', 'phecode_052-4': 'phecode_052-4 - Infectious mononucleosis', 'phecode_413-21': 'phecode_413-21 - Aortic stenosis', 'phecode_341': 'phecode_341 - Cerebral palsy and other paralytic syndromes', 'phecode_410': 'phecode_410 - Inflammation of the heart [Carditis]', 'phecode_463-1': 'phecode_463-1 - Chronic rhinitis', 'phecode_717-2': 'phecode_717-2 - Sacrococcygeal disorders', 'phecode_204': 'phecode_204 - Elevated blood glucose level', 'phecode_619-4': 'phecode_619-4 - Noninflammatory disorders of vagina', 'phecode_328-9': 'phecode_328-9 - Dementia NOS', 'phecode_256-2': 'phecode_256-2 - Hyposmolality and-or hyponatremia', 'phecode_425': 'phecode_425 - Cardiomegaly', 'phecode_504-1': 'phecode_504-1 - Gingivitis', 'phecode_256-7': 'phecode_256-7 - Volume depletion', 'phecode_479-3': 'phecode_479-3 - Respiratory failure', 'phecode_581-3': 'phecode_581-3 - Obstructive and reflux uropathy', 'phecode_704': 'phecode_704 - Systemic vasculitis', 'phecode_723-2': 'phecode_723-2 - Rotator cuff tear or rupture', 'phecode_464': 'phecode_464 - Nasopharyngitis', 'phecode_121-12': 'phecode_121-12 - Acute myeloid leukemia', 'phecode_464-1': 'phecode_464-1 - Acute nasopharyngitis', 'phecode_438': 'phecode_438 - Aneurysm or ectasia', 'phecode_200-3': 'phecode_200-3 - Thyrotoxicosis [hyperthyroidism]', 'phecode_283': 'phecode_283 - Other behavioral problems', 'phecode_016-1': 'phecode_016-1 - Chlamydia trachomatis', 'phecode_367-12': 'phecode_367-12 - Allergic [atopic] conjunctivitis', 'phecode_416-1': 'phecode_416-1 - Paroxysmal tachycardia', 'phecode_681': 'phecode_681 - Localized swelling, mass and lump of skin and subcutaneous tissue', 'phecode_391-2': 'phecode_391-2 - Eustachian tube disorders', 'phecode_136-2': 'phecode_136-2 - Benign neoplasm of stomach', 'phecode_542-2': 'phecode_542-2 - Fatty liver disease (FLD)', 'phecode_522-8': 'phecode_522-8 - Duodenitis', 'phecode_912': 'phecode_912 - Venous complications in pregnancy', 'phecode_413-6': 'phecode_413-6 - Heart valve replaced', 'phecode_554': 'phecode_554 - Diseases of the pancreas', 'phecode_684': 'phecode_684 - Diseases of hair and hair follicles', 'phecode_532': 'phecode_532 - Other disorders of the intestines', 'phecode_101-8': 'phecode_101-8 - Malignant neoplasm of the pancreas', 'phecode_546': 'phecode_546 - Other disorders of liver', 'phecode_397': 'phecode_397 - Other hearing abnormality', 'phecode_723-1': 'phecode_723-1 - Adhesive capsulitis of shoulder', 'phecode_413-3': 'phecode_413-3 - Tricuspid valve disorders', 'phecode_413-11': 'phecode_413-11 - Mitral valve insufficiency', 'phecode_666': 'phecode_666 - Urticaria', 'phecode_594-4': 'phecode_594-4 - Frequency of urination and polyuria', 'phecode_395-1': 'phecode_395-1 - Labyrinthitis', 'phecode_410-2': 'phecode_410-2 - Endocarditis', 'phecode_613-5': 'phecode_613-5 - Mastodynia', 'phecode_209': 'phecode_209 - Disorders of the pituitary gland and its hypothalamic control', 'phecode_513-1': 'phecode_513-1 - Esophageal ulcer', 'phecode_387': 'phecode_387 - Disorders of refraction and accommodation', 'phecode_374-4': 'phecode_374-4 - Retinal disorders in diseases classified elsewhere', 'phecode_002': 'phecode_002 - Staphylococcus', 'phecode_121-21': 'phecode_121-21 - Chronic lymphoid leukemia', 'phecode_281-1': 'phecode_281-1 - Substance abuse', 'phecode_397-1': 'phecode_397-1 - Tinnitus', 'phecode_334': 'phecode_334 - Disorders of other cranial nerves', 'phecode_287': 'phecode_287 - Psychotic disorder', 'phecode_684-1': 'phecode_684-1 - Alopecia', 'phecode_232-4': 'phecode_232-4 - Vitamin D deficiency', 'phecode_706-2': 'phecode_706-2 - Ankylosing spondylitis', 'phecode_367-2': 'phecode_367-2 - Keratitis', 'phecode_161': 'phecode_161 - Hemolytic anemias', 'phecode_726-2': 'phecode_726-2 - Pathologic fracture', 'phecode_074': 'phecode_074 - Aspergillosis', 'phecode_989': 'phecode_989 - Family history of respiratory disease', 'phecode_474-1': 'phecode_474-1 - Emphysema', 'phecode_389': 'phecode_389 - Other disorders of eye', 'phecode_618-2': 'phecode_618-2 - Uterine-Uterovaginal prolapse', 'phecode_722-4': 'phecode_722-4 - Palmar fascial fibromatosis [Dupuytren]', 'phecode_280-11': 'phecode_280-11 - Alcohol abuse', 'phecode_394-2': 'phecode_394-2 - Vertigo', 'phecode_686-2': 'phecode_686-2 - Non-pressure chronic ulcer', 'phecode_618-11': 'phecode_618-11 - Cystocele', 'phecode_622-1': 'phecode_622-1 - Polyp of corpus uteri', 'phecode_360-4': 'phecode_360-4 - Blepharitis', 'phecode_019': 'phecode_019 - Treponema', 'phecode_721-2': 'phecode_721-2 - Ganglion cyst', 'phecode_122-22': 'phecode_122-22 - Diffuse large B-cell lymphoma*', 'phecode_581-31': 'phecode_581-31 - Hydronephrosis', 'phecode_593-3': 'phecode_593-3 - Recurrent and persistent hematuria*', 'phecode_487-1': 'phecode_487-1 - Epistaxis', 'phecode_161-2': 'phecode_161-2 - Extrinsic (acquired) hemolytic anemias', 'phecode_234': 'phecode_234 - Other nutritional deficiencies', 'phecode_626-14': 'phecode_626-14 - Irregular menstrual bleeding', 'phecode_723-6': 'phecode_723-6 - Impingement syndrome of shoulder*', 'phecode_169': 'phecode_169 - Platelet defects', 'phecode_525-1': 'phecode_525-1 - Celiac disease', 'phecode_900': 'phecode_900 - Spontaneous abortion or history of recurrent pregnancy loss', 'phecode_169-1': 'phecode_169-1 - Thrombocytopenia', 'phecode_513-2': 'phecode_513-2 - Gastric ulcer', 'phecode_900-1': 'phecode_900-1 - Spontaneous abortion', 'phecode_465-11': 'phecode_465-11 - Streptococcal tonsillitis-pharyngitis', 'phecode_805': 'phecode_805 - Fever of unknown origin', 'phecode_333-2': 'phecode_333-2 - Insomnia', 'phecode_660-23': 'phecode_660-23 - Cutaneous erysipeloid', 'phecode_448-1': "phecode_448-1 - Raynaud's syndrome", 'phecode_280-12': 'phecode_280-12 - Alcohol dependence', 'phecode_135': 'phecode_135 - Benign neoplasm of the head and neck', 'phecode_542-1': 'phecode_542-1 - Fibrosis and cirrhosis of liver', 'phecode_668-2': 'phecode_668-2 - Seborrheic dermatitis', 'phecode_003': 'phecode_003 - Escherichia coli', 'phecode_376': 'phecode_376 - Disorders of vitreous body', 'phecode_726-1': 'phecode_726-1 - Osteoporosis', 'phecode_398': 'phecode_398 - Other disorders of ear', 'phecode_719-1': 'phecode_719-1 - Cramp and spasm', 'phecode_164-2': 'phecode_164-2 - Macrocytic anemia', 'phecode_416-42': 'phecode_416-42 - Left bundle branch block', 'phecode_556': 'phecode_556 - Other symptoms involving the digestive system and abdomen', 'phecode_349': 'phecode_349 - Disorder of nervous system', 'phecode_513-3': 'phecode_513-3 - Duodenal ulcer', 'phecode_713-2': 'phecode_713-2 - Effusion of joint', 'phecode_628': 'phecode_628 - Ovarian cyst', 'phecode_160-2': 'phecode_160-2 - Megaloblastic anemia', 'phecode_535': 'phecode_535 - Intestinal or peritoneal adhesions', 'phecode_101-1': 'phecode_101-1 - Malignant neoplasm of the esophagus', 'phecode_446-2': 'phecode_446-2 - Orthostatic hypotension', 'phecode_668-3': 'phecode_668-3 - Contact dermatitis', 'phecode_416-212': 'phecode_416-212 - Persistent atrial fibrillation*', 'phecode_351-3': 'phecode_351-3 - Paresthesia of skin*', 'phecode_479-6': 'phecode_479-6 - Pulmonary collapse [Atelectasis]', 'phecode_379': 'phecode_379 - Infection of the eye', 'phecode_103-1': 'phecode_103-1 - Melanomas of skin', 'phecode_594-2': 'phecode_594-2 - Dysuria', 'phecode_830': 'phecode_830 - Proteinuria', 'phecode_200-2': 'phecode_200-2 - Goiter', 'phecode_554-1': 'phecode_554-1 - Pancreatitis', 'phecode_286-1': 'phecode_286-1 - Bipolar disorder', 'phecode_084-2': 'phecode_084-2 - Malaria [Plasmodium]', 'phecode_584-1': 'phecode_584-1 - Renal dialysis', 'phecode_686-1': 'phecode_686-1 - Pressure ulcer', 'phecode_417-3': 'phecode_417-3 - Bradycardia*', 'phecode_416-41': 'phecode_416-41 - Atrioventricular block', 'phecode_660-21': 'phecode_660-21 - Impetigo', 'phecode_520-13': 'phecode_520-13 - Umbilical hernia', 'phecode_100': 'phecode_100 - Malignant neoplasm of the head and neck', 'phecode_495': 'phecode_495 - Abnormal findings on diagnostic imaging of lung', 'phecode_840-2': 'phecode_840-2 - Allergy to insects', 'phecode_376-2': 'phecode_376-2 - Vitreous opacities', 'phecode_256-5': 'phecode_256-5 - Hypokalemia [Hypopotassemia]', 'phecode_626-13': 'phecode_626-13 - Irregular menstrual cycle', 'phecode_394-21': 'phecode_394-21 - Paroxysmal vertigo', 'phecode_214': 'phecode_214 - Ovarian dysfunction', 'phecode_361': 'phecode_361 - Disorders of eyelid function', 'phecode_175': 'phecode_175 - Polycythemias', 'phecode_395-3': 'phecode_395-3 - Noise effects on inner ear', 'phecode_416-213': 'phecode_416-213 - Chronic atrial fibrillation*', 'phecode_938': 'phecode_938 - Other conditions or status of the mother complicating pregnancy, childbirth, or the puerperium', 'phecode_168': 'phecode_168 - Coagulation defects, purpura and other hemorrhagic conditions', 'phecode_161-24': 'phecode_161-24 - Paroxysmal nocturnal hemoglobinuria [Marchiafava-Micheli]*', 'phecode_341-2': 'phecode_341-2 - Hemiplegia and hemiparesis', 'phecode_718-1': 'phecode_718-1 - Radiculopathy', 'phecode_912-1': 'phecode_912-1 - Varicose veins in pregnancy', 'phecode_488-6': 'phecode_488-6 - Wheezing', 'phecode_438-1': 'phecode_438-1 - Aortic aneurysm and ectasia', 'phecode_726-5': 'phecode_726-5 - Osteomalacia', 'phecode_122-21': 'phecode_122-21 - Follicular lymphoma', 'phecode_416-11': 'phecode_416-11 - Supraventricular tachycardia', 'phecode_530-1': 'phecode_530-1 - Anal fissure', 'phecode_721-11': 'phecode_721-11 - Trigger finger', 'phecode_280-13': 'phecode_280-13 - Alcoholic liver disease', 'phecode_552': 'phecode_552 - Other diseases of biliary tract', 'phecode_144-1': 'phecode_144-1 - Benign neoplasms of external female genital organs and cervix', 'phecode_177-2': 'phecode_177-2 - Enlargement of lymph nodes [Lymphadenopathy]', 'phecode_124-6': 'phecode_124-6 - Myelodysplastic syndrome', 'phecode_360-11': 'phecode_360-11 - Hordeolum externum', 'phecode_360-1': 'phecode_360-1 - Hordeolum', 'phecode_708-12': 'phecode_708-12 - Primary osteoarthritis of knee, lower leg', 'phecode_374-3': 'phecode_374-3 - Retinal vascular changes and occlusions', 'phecode_232-27': 'phecode_232-27 - Vitamin B12 deficiency', 'phecode_618-12': 'phecode_618-12 - Rectocele', 'phecode_627-3': 'phecode_627-3 - Postmenopausal atrophic vaginitis', 'phecode_379-2': 'phecode_379-2 - Eye infection, viral', 'phecode_430': 'phecode_430 - Nontraumatic Intracranial hemorrhage', 'phecode_620': 'phecode_620 - Dysplasia of female genital organs', 'phecode_418': 'phecode_418 - Abnormal results of cardiovascular function studies', 'phecode_135-5': 'phecode_135-5 - Benign neoplasm of the paranasal sinus and nasal cavity', 'phecode_712': 'phecode_712 - Other specific joint derangements', 'phecode_710': 'phecode_710 - Acquired deformities of limbs', 'phecode_628-2': 'phecode_628-2 - Corpus luteum cyst or hematoma', 'phecode_688': 'phecode_688 - Granulomatous disorder of the skin', 'phecode_997': 'phecode_997 - Family history of other condition', 'phecode_471-5': 'phecode_471-5 - Nasal polyps', 'phecode_529-2': 'phecode_529-2 - Abdominal distension and flatulence', 'phecode_555': 'phecode_555 - Ascites', 'phecode_683-1': 'phecode_683-1 - Ingrowing nail', 'phecode_680': 'phecode_680 - Epidermal thickening', 'phecode_580': 'phecode_580 - Glomerular diseases', 'phecode_597-1': 'phecode_597-1 - Urethral stricture', 'phecode_324-3': 'phecode_324-3 - Dystonia', 'phecode_416-43': 'phecode_416-43 - Right bundle branch block', 'phecode_328-1': "phecode_328-1 - Alzheimer's disease", 'phecode_705-2': 'phecode_705-2 - Juvenile rheumatoid arthritis [JRA-JIA]', 'phecode_515': 'phecode_515 - Heartburn and epigastric pain', 'phecode_229': 'phecode_229 - Other endocrine disorders', 'phecode_514-2': 'phecode_514-2 - Intestinal obstruction', 'phecode_619-5': 'phecode_619-5 - Noninflammatory disorders of vulva and perineum', 'phecode_930': 'phecode_930 - Maternal care for abnormality of pelvic organs', 'phecode_507-2': 'phecode_507-2 - Cellulitis and abscess of mouth', 'phecode_807-1': 'phecode_807-1 - Chronic fatigue syndrome', 'phecode_807-11': 'phecode_807-11 - Postviral fatigue syndrome*', 'phecode_728': 'phecode_728 - Chondropathies', 'phecode_682-4': 'phecode_682-4 - Acne', 'phecode_491': 'phecode_491 - Pleurisy', 'phecode_106-2': 'phecode_106-2 - Malignant neoplasm of the uterus', 'phecode_179': 'phecode_179 - Immunodeficiencies', 'phecode_200-12': 'phecode_200-12 - Hypothyroidism due to drugs or iatrogenic causes', 'phecode_417-2': 'phecode_417-2 - Tachycardia', 'phecode_396-2': 'phecode_396-2 - Sensorineural hearing loss', 'phecode_146': 'phecode_146 - Benign neoplasm of the genitourinary system', 'phecode_431-2': 'phecode_431-2 - Transient cerebral ischemic attacks', 'phecode_360-2': 'phecode_360-2 - Chalazion', 'phecode_486-5': 'phecode_486-5 - Abnormal sputum', 'phecode_116-5': 'phecode_116-5 - Secondary malignancy of brain-spine', 'phecode_377': 'phecode_377 - Hemorrhage of the eye', 'phecode_520-14': 'phecode_520-14 - Ventral hernia', 'phecode_367-13': 'phecode_367-13 - Blepharoconjunctivitis', 'phecode_348': 'phecode_348 - Other diseases of spinal cord', 'phecode_615': 'phecode_615 - Endometriosis', 'phecode_540': 'phecode_540 - Hepatitis', 'phecode_320': 'phecode_320 - Meningitis', 'phecode_084-1': 'phecode_084-1 - Amebiasis [Entamoeba histolytica]', 'phecode_200-6': 'phecode_200-6 - Atrophy of thyroid*', 'phecode_200-15': 'phecode_200-15 - Postinfectious hypothyroidism*', 'phecode_714': 'phecode_714 - Deforming dorsopathies', 'phecode_841-3': 'phecode_841-3 - Allergy to narcotic agent', 'phecode_033': 'phecode_033 - Francisella', 'phecode_406': 'phecode_406 - Chronic pulmonary heart disease', 'phecode_610-1': 'phecode_610-1 - Cystic mastopathy', 'phecode_684-13': 'phecode_684-13 - Scarring hair loss*', 'phecode_529-3': 'phecode_529-3 - Fecal incontinence', 'phecode_374-1': 'phecode_374-1 - Retinal detachments and breaks', 'phecode_139-53': 'phecode_139-53 - Lipoma of other skin subcutaneous tissue', 'phecode_832': 'phecode_832 - Other abnormal findings in urine', 'phecode_247-5': 'phecode_247-5 - Disorders of calcium metabolism', 'phecode_320-1': 'phecode_320-1 - Infective meningitis', 'phecode_108-4': 'phecode_108-4 - Malignant neoplasm of the kidney', 'phecode_002-1': 'phecode_002-1 - Staphylococcus aureus', 'phecode_180': 'phecode_180 - Other disorders involving the immune mechanism', 'phecode_101-2': 'phecode_101-2 - Malignant neoplasm of stomach', 'phecode_723-5': 'phecode_723-5 - Tendinitis', 'phecode_431-12': 'phecode_431-12 - Hemorrhagic stroke', 'phecode_831': 'phecode_831 - Glycosuria', 'phecode_106-21': 'phecode_106-21 - Malignant neoplasm of endometrium', 'phecode_550-2': 'phecode_550-2 - Cholecystitis', 'phecode_700-1': 'phecode_700-1 - Lupus', 'phecode_626-11': 'phecode_626-11 - Absent or infrequent menstruation', 'phecode_200-16': 'phecode_200-16 - Myxedema*', 'phecode_557-2': 'phecode_557-2 - Blood in stool', 'phecode_413-22': 'phecode_413-22 - Aortic insufficiency', 'phecode_601': 'phecode_601 - Inflammatory diseases of prostate', 'phecode_554-11': 'phecode_554-11 - Acute pancreatitis', 'phecode_468-1': 'phecode_468-1 - Viral pneumonia', 'phecode_596-5': 'phecode_596-5 - Neuromuscular dysfunction of bladder', 'phecode_674': 'phecode_674 - Disorders of pigmentation', 'phecode_700-2': 'phecode_700-2 - Sicca syndrome [Sjogren syndrome]', 'phecode_601-1': 'phecode_601-1 - Prostatitis', 'phecode_109': 'phecode_109 - Malignant neoplasm of the eye, brain and other parts of central nervous system', 'phecode_979-2': 'phecode_979-2 - Kidney transplant', 'phecode_375-11': 'phecode_375-11 - Open angle glaucoma', 'phecode_835': 'phecode_835 - Cytology and pathology findings', 'phecode_901': 'phecode_901 - Abnormal products of conception', 'phecode_664': 'phecode_664 - Papulosquamous disorders', 'phecode_162': 'phecode_162 - Aplastic anemia', 'phecode_369': 'phecode_369 - Noninflammatory disorders of the cornea', 'phecode_406-1': 'phecode_406-1 - Pulmonary hypertension', 'phecode_144-13': 'phecode_144-13 - Benign neoplasms of the cervix', 'phecode_208': 'phecode_208 - Disorders of parathyroid gland', 'phecode_586-2': 'phecode_586-2 - Cyst of kidney', 'phecode_438-11': 'phecode_438-11 - Abdominal aortic aneurysm', 'phecode_494': 'phecode_494 - Voice disturbance', 'phecode_001': 'phecode_001 - Salmonella', 'phecode_059': 'phecode_059 - Coronavirus', 'phecode_108-41': 'phecode_108-41 - Malignant neoplasm of kidney, except pelvis', 'phecode_724-5': 'phecode_724-5 - Exostosis', 'phecode_164-6': 'phecode_164-6 - Anemia secondary to chronic diseases and conditions', 'phecode_723-51': 'phecode_723-51 - Achilles tendinitis', 'phecode_704-5': 'phecode_704-5 - Giant cell arteritis', 'phecode_059-1': 'phecode_059-1 - COVID-19*', 'phecode_356': 'phecode_356 - Speech disturbance', 'phecode_556-8': 'phecode_556-8 - Nonspecific abnormal findings in stool contents', 'phecode_701': 'phecode_701 - Osteomyelitis, periostitis, and other infections involving bone', 'phecode_440-2': 'phecode_440-2 - Arterial embolism and thrombosis', 'phecode_507-1': 'phecode_507-1 - Stomatitis', 'phecode_347': 'phecode_347 - Other disorders of the brain and CNS', 'phecode_708-11': 'phecode_708-11 - Primary osteoarthritis of hip, pelvic region and thigh', 'phecode_488-8': 'phecode_488-8 - Mouth breathing*', 'phecode_585-1': 'phecode_585-1 - Renal colic', 'phecode_700-11': 'phecode_700-11 - Systemic lupus erythematosus [SLE]', 'phecode_622-2': 'phecode_622-2 - Mucous polyp of cervix', 'phecode_471-2': 'phecode_471-2 - Deviated nasal septum', 'phecode_120-13': 'phecode_120-13 - Erythroid', 'phecode_324-4': 'phecode_324-4 - Tremor', 'phecode_337-2': 'phecode_337-2 - Inflammatory polyneuropathy', 'phecode_626-2': 'phecode_626-2 - Dysmenorrhea', 'phecode_514-1': 'phecode_514-1 - Esophageal obstruction (Stricture and stenosis of esophagus)', 'phecode_367-5': 'phecode_367-5 - Uveitis', 'phecode_381': 'phecode_381 - Strabismus', 'phecode_682-11': 'phecode_682-11 - Sebaceous cyst [Epidermal cyst]', 'phecode_367-52': 'phecode_367-52 - Iridocyclitis', 'phecode_714-3': 'phecode_714-3 - Scoliosis', 'phecode_208-2': 'phecode_208-2 - Hyperparathyroidism', 'phecode_503': 'phecode_503 - Diseases of pulp and periapical tissues', 'phecode_109-3': 'phecode_109-3 - Malignant neoplasm of brain', 'phecode_214-1': 'phecode_214-1 - Primary ovarian failure', 'phecode_526-3': 'phecode_526-3 - Intestinal infection due to protozoa', 'phecode_256-3': 'phecode_256-3 - Mixed disorder of acid-base balance', 'phecode_334-1': 'phecode_334-1 - Trigeminal nerve disorders [CN5]', 'phecode_387-2': 'phecode_387-2 - Myopia', 'phecode_522-91': 'phecode_522-91 - Atrophic gastritis', 'phecode_104': 'phecode_104 - Malignant sarcoma-related cancers', 'phecode_205': 'phecode_205 - Hypoglycemia', 'phecode_391-12': 'phecode_391-12 - Chronic otitis media', 'phecode_501': 'phecode_501 - Dental caries', 'phecode_124-3': 'phecode_124-3 - Polycythemia vera', 'phecode_581-1': 'phecode_581-1 - Pyelonephritis', 'phecode_288-4': 'phecode_288-4 - Phobic disorders', 'phecode_443': 'phecode_443 - Other specified disorders of arteries and arterioles', 'phecode_377-2': 'phecode_377-2 - Conjunctival hemorrhage', 'phecode_388': 'phecode_388 - Blindness and low vision', 'phecode_288-2': 'phecode_288-2 - Panic disorder [episodic paroxysmal anxiety]', 'phecode_685': 'phecode_685 - Disorders of sweat glands', 'phecode_724-52': 'phecode_724-52 - Osteophyte*', 'phecode_503-5': 'phecode_503-5 - Periapical abscess', 'phecode_614-52': 'phecode_614-52 - Vaginitis and vulvovaginitis', 'phecode_468-2': 'phecode_468-2 - Bacterial pneumonia', 'phecode_557-1': 'phecode_557-1 - Hematemesis', 'phecode_688-1': 'phecode_688-1 - Sarcoidosis', 'phecode_348-2': 'phecode_348-2 - Myelopathies', 'phecode_506': 'phecode_506 - Diseases of salivary glands', 'phecode_526-13': 'phecode_526-13 - Shigellosis', 'phecode_027': 'phecode_027 - Shigella', 'phecode_284-2': 'phecode_284-2 - Suicide and self-inflicted harm', 'phecode_284': 'phecode_284 - Suicide ideation and attempt or self harm', 'phecode_256-4': 'phecode_256-4 - Hyperkalemia [Hyperpotassemia]', 'phecode_174': 'phecode_174 - Diseases of spleen', 'phecode_715-3': 'phecode_715-3 - Spondylolisthesis', 'phecode_619-3': 'phecode_619-3 - Noninflammatory disorders of cervix', 'phecode_540-1': 'phecode_540-1 - Chronic hepatitis', 'phecode_530-2': 'phecode_530-2 - Anorectal abscess', 'phecode_619-1': 'phecode_619-1 - Noninflammatory disorders of ovary, fallopian tube, and broad ligament', 'phecode_287-5': 'phecode_287-5 - Drug-induced psychotic disorder', 'phecode_023': 'phecode_023 - Vibrio', 'phecode_558': 'phecode_558 - Abnormal findings on diagnostic imaging of the digestive tract', 'phecode_146-2': 'phecode_146-2 - Benign neoplasm of the prostate', 'phecode_804': 'phecode_804 - Other symptoms and signs involving the circulatory and respiratory system', 'phecode_508': 'phecode_508 - Diseases of lips', 'phecode_391-11': 'phecode_391-11 - Acute otitis media', 'phecode_256-31': 'phecode_256-31 - Acidosis', 'phecode_287-1': 'phecode_287-1 - Schizophrenia', 'phecode_700-3': 'phecode_700-3 - Systemic sclerosis', 'phecode_140': 'phecode_140 - Benign neoplasm of the breast', 'phecode_139-6': 'phecode_139-6 - Hemangioma and lymphangioma', 'phecode_580-3': 'phecode_580-3 - Nephrotic syndrome', 'phecode_478': 'phecode_478 - Aspiration pneumonia', 'phecode_343': 'phecode_343 - Disorders of autonomic nervous system', 'phecode_341-1': 'phecode_341-1 - Cerebral palsy', 'phecode_180-3': 'phecode_180-3 - Paraproteinemias', 'phecode_329-1': 'phecode_329-1 - Memory loss', 'phecode_540-3': 'phecode_540-3 - Viral hepatitits', 'phecode_054': 'phecode_054 - Hepatovirus', 'phecode_665-2': 'phecode_665-2 - Psoriatic arthropathy', 'phecode_330-11': 'phecode_330-11 - Generalized epilepsy', 'phecode_394-1': 'phecode_394-1 - Meniere disease', 'phecode_284-29': 'phecode_284-29 - Intentional self-harm*', 'phecode_664-1': 'phecode_664-1 - Lichen planus, nitidus, or striatus', 'phecode_168-1': 'phecode_168-1 - Hypo-coagulability', 'phecode_139-61': 'phecode_139-61 - Hemangioma', 'phecode_467-11': 'phecode_467-11 - Acute obstructive laryngitis [croup]', 'phecode_334-11': 'phecode_334-11 - Trigeminal neuralgia', 'phecode_351-2': 'phecode_351-2 - Hypoesthesia of skin*', 'phecode_247-4': 'phecode_247-4 - Disorders of magnesium metabolism', 'phecode_537': 'phecode_537 - Abnormality of the peritoneum', 'phecode_423': 'phecode_423 - Abnormal cardiac sounds', 'phecode_211': 'phecode_211 - Disorders of adrenal glands', 'phecode_423-1': 'phecode_423-1 - Cardiac murmurs', 'phecode_098': 'phecode_098 - Carrier or suspected carrier of infectious diseases', 'phecode_675': 'phecode_675 - Atrophic conditions of skin', 'phecode_334-2': 'phecode_334-2 - Facial nerve disorders and weakness', 'phecode_106-1': 'phecode_106-1 - Malignant neoplasm of external female genital organs and cervix', 'phecode_177-4': 'phecode_177-4 - Lymphedema', 'phecode_256-6': 'phecode_256-6 - Fluid overload', 'phecode_244': 'phecode_244 - Disorders of lipoprotein metabolism and other lipidemias', 'phecode_327': 'phecode_327 - Other degenerative diseases of nervous system', 'phecode_701-1': 'phecode_701-1 - Osteomyelitis', 'phecode_675-1': 'phecode_675-1 - Circumscribed scleroderma', 'phecode_624': 'phecode_624 - Symptoms involving female genital tract', 'phecode_729': 'phecode_729 - Other acquired musculoskeletal deformity', 'phecode_148': 'phecode_148 - Benign neoplasm of the eye, brain and other parts of central nervous system', 'phecode_596-3': 'phecode_596-3 - Diverticulum of bladder', 'phecode_101-6': 'phecode_101-6 - Malignant neoplasm of the liver and intrahepatic bile ducts', 'phecode_331-1': 'phecode_331-1 - Tension headache', 'phecode_723-3': "phecode_723-3 - Medial epicondylitis (Golfer's elbow)", 'phecode_613-1': 'phecode_613-1 - Inflammatory disease of breast', 'phecode_619-2': 'phecode_619-2 - Disorders of uterus, NEC', 'phecode_387-3': 'phecode_387-3 - Astigmatism', 'phecode_848': 'phecode_848 - Nonspecific abnormal findings of other body structures', 'phecode_603': 'phecode_603 - Disorders and symptoms of testis', 'phecode_532-1': 'phecode_532-1 - Intestinal fistula', 'phecode_416-5': 'phecode_416-5 - Premature depolarization [Premature beats]', 'phecode_344': 'phecode_344 - Disorders of the circulation of the cerebrospinal fluid', 'phecode_721-6': "phecode_721-6 - Baker's cyst [popliteal cyst]", 'phecode_336-55': 'phecode_336-55 - Lesion of plantar nerve', 'phecode_324-34': 'phecode_324-34 - Torticollis', 'phecode_449': 'phecode_449 - Other disorders of the circulatory system', 'phecode_509': 'phecode_509 - Diseases of tongue', 'phecode_667': 'phecode_667 - Erythematous conditions', 'phecode_169-11': 'phecode_169-11 - Immune thrombocytopenic purpura [ITP]', 'phecode_200-4': 'phecode_200-4 - Thyroiditis', 'phecode_411': 'phecode_411 - Other diseases of pericardium', 'phecode_430-2': 'phecode_430-2 - Nontraumatic intracerebral hemorrhage', 'phecode_556-3': 'phecode_556-3 - Abdominal or pelvic swelling, mass, or lump', 'phecode_912-2': 'phecode_912-2 - Thrombotic conditions of pregnancy', 'phecode_700-4': 'phecode_700-4 - Inflammatory myopathy', 'phecode_542-4': 'phecode_542-4 - Portal hypertension', 'phecode_299': 'phecode_299 - Mental disorder, not otherwise specified', 'phecode_537-1': 'phecode_537-1 - Peritonitis', 'phecode_414-2': 'phecode_414-2 - Dilated cardiomyopathy*', 'phecode_440-11': 'phecode_440-11 - Deep vein thrombosis [DVT]', 'phecode_165': 'phecode_165 - Hemoglobinopathies', 'phecode_086': 'phecode_086 - Pediculosis, acariasis and other infestations', 'phecode_091': 'phecode_091 - Gangrene', 'phecode_596-1': 'phecode_596-1 - Bladder neck obstruction', 'phecode_614-54': 'phecode_614-54 - Abscess or ulceration of vulva', 'phecode_661': 'phecode_661 - Viral exanthemata NOS', 'phecode_702-3': 'phecode_702-3 - Enteropathic arthropathies', 'phecode_676-2': 'phecode_676-2 - Scar conditions and fibrosis of skin', 'phecode_674-2': 'phecode_674-2 - Hypepigmentation', 'phecode_720': 'phecode_720 - Spontaneous rupture of synovium and tendon', 'phecode_122-1': 'phecode_122-1 - Hodgkin lymphoma', 'phecode_009': 'phecode_009 - Pseudomonas', 'phecode_430-1': 'phecode_430-1 - Nontraumatic subarachnoid hemorrhage', 'phecode_554-12': 'phecode_554-12 - Chronic pancreatitis', 'phecode_433-2': 'phecode_433-2 - Occlusion and stenosis of precerebral arteries', 'phecode_709-24': 'phecode_709-24 - Hammer toe', 'phecode_380': 'phecode_380 - Disorders of optic nerve and visual pathways', 'phecode_603-1': 'phecode_603-1 - Hydrocele', 'phecode_361-3': 'phecode_361-3 - Ptosis of eyelid', 'phecode_730': 'phecode_730 - Other disorders and symptoms of the musculoskeletal system', 'phecode_200-21': 'phecode_200-21 - Diffuse goiter', 'phecode_323': 'phecode_323 - Systemic atrophies primarily affecting the central nervous system', 'phecode_391-7': 'phecode_391-7 - Perforation of tympanic membrane', 'phecode_954': 'phecode_954 - Other disorders of the breast associated with childbirth and disorders of lactation', 'phecode_416-12': 'phecode_416-12 - Ventricular tachycardia', 'phecode_680-1': 'phecode_680-1 - Corns and callosities', 'phecode_334-21': "phecode_334-21 - Bell's palsy", 'phecode_625-2': 'phecode_625-2 - Postcoital bleeding', 'phecode_112-1': 'phecode_112-1 - Mesothelioma*', 'phecode_366': 'phecode_366 - Noninflammatory disorders of conjunctiva', 'phecode_433-1': 'phecode_433-1 - Occlusion and stenosis of cerebral arteries', 'phecode_324-41': 'phecode_324-41 - Essential tremor*', 'phecode_484': 'phecode_484 - Pneumothorax and air leak', 'phecode_911': 'phecode_911 - Excessive vomiting in pregnancy', 'phecode_396-21': 'phecode_396-21 - Sensorineural hearing loss, bilateral', 'phecode_400-1': 'phecode_400-1 - Acute rheumatic fever', 'phecode_433-21': 'phecode_433-21 - Carotid artery stenosis', 'phecode_414-5': 'phecode_414-5 - Ischemic cardiomyopathy*', 'phecode_848-2': 'phecode_848-2 - Nonspecific abnormal findings on radiological and other examination of other intrathoracic organs (echo)', 'phecode_363-5': 'phecode_363-5 - Epiphora', 'phecode_614-1': 'phecode_614-1 - Pelvic peritoneal adhesions, female (postoperative) (postinfection)', 'phecode_175-2': 'phecode_175-2 - Secondary polycythemia', 'phecode_486-2': 'phecode_486-2 - Other diseases of bronchus', 'phecode_624-1': 'phecode_624-1 - Dystrophy of female genital tract', 'phecode_464-2': 'phecode_464-2 - Chronic nasopharyngitis', 'phecode_230-3': 'phecode_230-3 - Anorexia', 'phecode_709-22': 'phecode_709-22 - Hallux rigidus', 'phecode_660-7': 'phecode_660-7 - Pilonidal cyst', 'phecode_629': 'phecode_629 - Female infertility', 'phecode_211-2': 'phecode_211-2 - Adrenocortical insufficiency', 'phecode_007': 'phecode_007 - Hemophilus infection', 'phecode_679-2': 'phecode_679-2 - Pallor and flushing', 'phecode_341-12': 'phecode_341-12 - Athetoid cerebral palsy', 'phecode_552-1': 'phecode_552-1 - Cholangitis', 'phecode_718-3': 'phecode_718-3 - Mid back pain', 'phecode_179-1': 'phecode_179-1 - Hypogammaglobulinemia NOS', 'phecode_106-13': 'phecode_106-13 - Malignant neoplasm of the cervix', 'phecode_518': 'phecode_518 - Appendicitis', 'phecode_336-2': 'phecode_336-2 - Lesion of median, ulnar, radial nerve', 'phecode_592-11': 'phecode_592-11 - Acute cystitis', 'phecode_144-3': 'phecode_144-3 - Benign neoplasms of the ovary', 'phecode_149': 'phecode_149 - Benign neoplasm of the endocrine glands', 'phecode_945': 'phecode_945 - Rupture of uterus', 'phecode_100-1': 'phecode_100-1 - Malignant neoplasm of the oral cavity', 'phecode_945-3': 'phecode_945-3 - Postpartum hemorrhage', 'phecode_552-2': 'phecode_552-2 - Obstruction of bile duct', 'phecode_004-1': 'phecode_004-1 - Streptococcus pneumoniae', 'phecode_138-1': 'phecode_138-1 - Nevus, non-neoplastic', 'phecode_447': 'phecode_447 - Nonspecific low blood-pressure reading', 'phecode_209-2': 'phecode_209-2 - Hypopituitarism', 'phecode_308-4': 'phecode_308-4 - Demoralization and apathy', 'phecode_728-3': "phecode_728-3 - Costochondritis (Tietze's disease)", 'phecode_466-2': 'phecode_466-2 - Chronic tonsillitis and adenoiditis', 'phecode_679-22': 'phecode_679-22 - Flushing', 'phecode_177-1': 'phecode_177-1 - Lymphadenitis', 'phecode_353': 'phecode_353 - Symptoms and signs involving general sensations and perceptions', 'phecode_414-1': 'phecode_414-1 - Hypertrophic cardiomyopathy', 'phecode_283-4': "phecode_283-4 - Patient's noncompliance with medical treatment and regimen", 'phecode_101-21': 'phecode_101-21 - Malignant neoplasm of cardia', 'phecode_949': 'phecode_949 - Complications of labor and delivery NEC', 'phecode_904': 'phecode_904 - Bleeding in pregnancy', 'phecode_542-3': 'phecode_542-3 - Hepatic failure', 'phecode_817': 'phecode_817 - Motion sickness', 'phecode_105-1': 'phecode_105-1 - Malignant neoplasm of the breast, female', 'phecode_668-5': 'phecode_668-5 - Lichen simplex chronicus', 'phecode_200-23': 'phecode_200-23 - Multinodular goiter', 'phecode_614-51': 'phecode_614-51 - Cervicitis and endocervicitis', 'phecode_901-3': 'phecode_901-3 - Missed abortion', 'phecode_342': 'phecode_342 - Plegia and unspecified paralysis', 'phecode_356-2': 'phecode_356-2 - Aphasia and dysphasia', 'phecode_370': 'phecode_370 - Disorders of iris and ciliary body', 'phecode_468-21': 'phecode_468-21 - Pneumococcal pneumonia', 'phecode_700-41': 'phecode_700-41 - Myositis', 'phecode_249': 'phecode_249 - Amyloidosis', 'phecode_363-6': 'phecode_363-6 - Inflammation of lacrimal passages', 'phecode_482': 'phecode_482 - Suppurative and necrotic conditions of the lower respiratory tract', 'phecode_030': 'phecode_030 - Campylobacter', 'phecode_052-31': 'phecode_052-31 - Varicella [chickenpox]', 'phecode_381-1': 'phecode_381-1 - Paralytic strabismus [Neurogenic strabismus]', 'phecode_625-1': 'phecode_625-1 - Dyspareunia', 'phecode_411-2': 'phecode_411-2 - Pericardial effusion (noninflammatory)*', 'phecode_374-11': 'phecode_374-11 - Serous retinal detachment', 'phecode_510-6': 'phecode_510-6 - Diverticulum of esophagus', 'phecode_180-31': 'phecode_180-31 - Monoclonal gammopathy', 'phecode_604-5': 'phecode_604-5 - Balanoposthitis', 'phecode_015': 'phecode_015 - Clostridium', 'phecode_712-6': 'phecode_712-6 - Instability of joint', 'phecode_614-3': 'phecode_614-3 - Pelvic inflammatory disease (PID)', 'phecode_174-1': 'phecode_174-1 - Hyposplenism*', 'phecode_200-31': "phecode_200-31 - Graves' disease [Toxic diffuse goiter]", 'phecode_376-21': 'phecode_376-21 - Crystalline deposits in vitreous body', 'phecode_660-11': 'phecode_660-11 - Candidiasis of skin and nails', 'phecode_386-2': 'phecode_386-2 - Diplopia', 'phecode_502': 'phecode_502 - Other diseases of teeth and supporting structures', 'phecode_200-44': 'phecode_200-44 - Chronic thyroiditis', 'phecode_500': 'phecode_500 - Disorders of tooth develoment', 'phecode_251': 'phecode_251 - Disorders of bilirubin excretion', 'phecode_702-1': 'phecode_702-1 - Pyogenic arthritis [Septic arthritis]', 'phecode_725': 'phecode_725 - Developmental disorders of bone', 'phecode_507-11': 'phecode_507-11 - Recurrent oral aphthae [Recurrent aphthous stomatitis]', 'phecode_592-13': 'phecode_592-13 - Chronic interstitial cystitis', 'phecode_007-1': 'phecode_007-1 - Hemophilus influenzae', 'phecode_618-5': 'phecode_618-5 - Prolapse of vaginal vault after hysterectomy', 'phecode_504-3': 'phecode_504-3 - Periodontitis', 'phecode_338': 'phecode_338 - Myasthenia gravis and other myoneural disorders', 'phecode_413-32': 'phecode_413-32 - Tricuspid value insufficiency*', 'phecode_725-2': 'phecode_725-2 - Juvenile osteochondrosis', 'phecode_343-1': 'phecode_343-1 - Autonomic neuropathy', 'phecode_704-1': 'phecode_704-1 - Polyarteritis nodosa and related conditions', 'phecode_530-3': 'phecode_530-3 - Rectal prolapse', 'phecode_355': 'phecode_355 - Coma and other alteration of consciousness', 'phecode_177-13': 'phecode_177-13 - Acute lymphadenitis', 'phecode_814': 'phecode_814 - Jaundice (not of newborn)', 'phecode_209-1': 'phecode_209-1 - Pituitary hyperfunction', 'phecode_707-8': 'phecode_707-8 - Polyarthritis', 'phecode_841-13': 'phecode_841-13 - Allergy to sulfonamides', 'phecode_251-1': 'phecode_251-1 - Gilbert syndrome*', 'phecode_619-32': 'phecode_619-32 - Erosion and ectropion of cervix', 'phecode_523-3': 'phecode_523-3 - Gastric diverticulum', 'phecode_350-1': 'phecode_350-1 - Tetany', 'phecode_437': 'phecode_437 - Vascular insufficiency of intestine', 'phecode_208-21': 'phecode_208-21 - Primary hyperparathyroidism', 'phecode_679-3': 'phecode_679-3 - Changes in skin texture', 'phecode_283-8': 'phecode_283-8 - Other problems related to lifestyle', 'phecode_482-2': 'phecode_482-2 - Pyothorax (empyema)', 'phecode_120-22': 'phecode_120-22 - Mature T-Cell', 'phecode_614-53': "phecode_614-53 - Cyst or abscess of Bartholin's gland", 'phecode_164-3': 'phecode_164-3 - Acute posthemorrhagic anemia'}
endpoints = sorted(endpoint_dict.keys())

In [5]:
covariates = ["age_at_recruitment_f21022_0_0", "sex_f31_0_0",  "ethnic_background_f21000_0_0"]

In [3]:
data_covariates = pa.feather.read_table("/sc-projects/sc-proj-ukb-cvd/data/2_datasets_pre/211110_anewbeginning/baseline_covariates_211209.feather", memory_map=True)

AttributeError: module 'pyarrow' has no attribute 'feather'

In [6]:
data_covariates = pd.read_feather("/sc-projects/sc-proj-ukb-cvd/data/2_datasets_pre/211110_anewbeginning/baseline_covariates_211209.feather")\
    .set_index("eid")[covariates]\
    .assign(age_at_recruitment_f21022_0_0 = lambda x: x.age_at_recruitment_f21022_0_0.astype(np.int32))

In [7]:
data_logh = pd.read_feather(f"{output_path}/predictions_{tag}/predictions_wide.feather").set_index("eid")

In [8]:
data_all = data_covariates.merge(data_logh, left_index=True, right_index=True, how="left")

In [ ]:
import pyarrow as pa
data_all_arrow = pa.Table.from_pandas(data_all)

In [12]:
data_covariates_ray = ray.put(data_covariates)

In [13]:
data_logh_ray = ray.put(data_logh)

In [14]:
def norm_variables(data_partition, variables):
    norm = StandardScaler(with_mean=True, with_std=True, copy=True).fit(data_partition.query("split=='train'")[variables].values)

    data_partition[variables] = norm.transform(data_partition[variables].values)
    
    return data_partition

In [15]:
@ray.remote
def merge_norm_df(data_covariates, data_logh, model, partition, variables):
    temp_logh = data_logh.query("model==@model").query("partition==@partition")
    temp_partition = temp_logh.merge(data_covariates, left_index=True, right_index=True, how="left")
    temp_partition_normed = norm_variables(data_partition, variables)
    return temp_partition_normed

In [16]:
def norm_logh_and_extra(data_covariates_ray, data_logh_ray, variables):
    dfs = []
    for model in tqdm(models):
        for partition in tqdm([int(p) for p in partitions]):
            data_partition = merge_norm_df.remote(data_covariates_ray, data_logh_ray, model, partition, variables)
            dfs.append(data_partition)
    dfs_finished = [ray.get(df) for df in tqdm(dfs)]
    return pd.concat(dfs_finished, axis=0)

In [17]:
models = data_logh.model.unique().tolist()
partitions = data_logh.partition.unique().tolist()
variables_to_norm = ["age_at_recruitment_f21022_0_0"] + endpoints

In [ ]:
data_all_normed = norm_logh_and_extra(data_covariates_ray, data_logh_ray, variables_to_norm)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/66 [00:00<?, ?it/s]

In [2]:
import pyarrow

In [3]:
pyarrow.__version__

'7.0.0'

In [4]:
pyarrow.dataset

AttributeError: module 'pyarrow' has no attribute 'dataset'

In [6]:
pyarrow.dataset(source=f"{output_path}/predictions_{tag}/predictions_wide.feather")

AttributeError: module 'pyarrow' has no attribute 'dataset'

In [9]:
pyarrow.__version__

'4.0.0'

In [ ]:
data_dict[list(data_dict.keys())[0]]

In [11]:
data_all_normed = norm_logh_and_extra(data_all, variables_to_norm)

  0%|          | 0/3 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [33]:
endpoint_defs = pd.read_feather(f"{output_path}/phecode_defs_220306.feather").query("endpoint==@endpoints").sort_values("endpoint").set_index("endpoint")

In [6]:
data_covariates = pd.read_feather("/sc-projects/sc-proj-ukb-cvd/data/2_datasets_pre/211110_anewbeginning/baseline_covariates_211209.feather")[["eid", "sex_f31_0_0"]].set_index("eid")

In [32]:
data_outcomes = pd.read_feather(f"{output_path}/baseline_outcomes_220223.feather").set_index("eid")

In [16]:
data_all = data_outcomes.merge(data_covariates, left_index=True, right_index=True, how="left").reset_index(drop=False).set_index("endpoint")

In [22]:
data_dict = {e: df.reset_index(drop=True).set_index("eid") for e, df in data_all.groupby('endpoint')}

In [41]:
data_temp

,prevalent,event,time,sex_f31_0_0
eid,,,,
1000018,0.0,0.0,11.359576,Female
1000020,0.0,0.0,13.089933,Male
1000037,0.0,0.0,12.361650,Female
1000043,0.0,0.0,11.803117,Male
1000051,0.0,0.0,14.784698,Female
...,...,...,...,...
6025150,0.0,0.0,13.730603,Female
6025165,0.0,0.0,12.553304,Female
6025173,0.0,0.0,12.512235,Male


In [40]:
endpoint_defs.sex.unique()

array(['Both', 'Female', 'Male'], dtype=object)

In [50]:
def get_eligable_eids(data_dict, endpoint):

    data_temp = data_dict[endpoint]
    eligibility = endpoint_defs.loc[endpoint]["sex"]
    
    if eligibility == "Both": 
        eids_incl = data_temp.copy().query(f"prevalent==0").index.to_list()
    else:
        eids_incl = data_temp.copy().query(f"prevalent==0&sex_f31_0_0==@eligibility").index.to_list()
        
    return {"endpoint": endpoint, 
            "n_eids": len(eids_incl), 
            "eid_list": eids_incl}

In [54]:
d_list = [get_eligable_eids(data_dict, endpoint) for endpoint in tqdm(endpoints)]
eid_df = pd.DataFrame.from_dict(d_list)

  0%|          | 0/1001 [00:02<?, ?it/s]

In [56]:
eid_df.to_feather(f"{output_path}/eligable_eids_220306.feather")